# Instalaciones necesarias

In [ ]:
!pip install cuda-python
# !pip install mpi4py
# !pip install nvidia-ml-py3

# Información de la GPU

In [ ]:
from numba import cuda
gpu = cuda.get_current_device()
print("name = %s" % gpu.name)
print("maxThreadsPerBlock = %s" % str(gpu.MAX_THREADS_PER_BLOCK))
print("maxBlockDimX = %s" % str(gpu.MAX_BLOCK_DIM_X))
print("maxBlockDimY = %s" % str(gpu.MAX_BLOCK_DIM_Y))
print("maxBlockDimZ = %s" % str(gpu.MAX_BLOCK_DIM_Z))
print("maxGridDimX = %s" % str(gpu.MAX_GRID_DIM_X))
print("maxGridDimY = %s" % str(gpu.MAX_GRID_DIM_Y))
print("maxGridDimZ = %s" % str(gpu.MAX_GRID_DIM_Z))
print("maxSharedMemoryPerBlock = %s" % str(gpu.MAX_SHARED_MEMORY_PER_BLOCK))
print("asyncEngineCount = %s" % str(gpu.ASYNC_ENGINE_COUNT))
print("canMapHostMemory = %s" % str(gpu.CAN_MAP_HOST_MEMORY))
print("multiProcessorCount = %s" % str(gpu.MULTIPROCESSOR_COUNT))
print("maxThreadsPerMultiprocessor = %s" % str(gpu.MAX_THREADS_PER_MULTI_PROCESSOR))
print("warpSize = %s" % str(gpu.WARP_SIZE))
print("unifiedAddressing = %s" % str(gpu.UNIFIED_ADDRESSING))
print("pciBusID = %s" % str(gpu.PCI_BUS_ID))
print("pciDeviceID = %s" % str(gpu.PCI_DEVICE_ID))

cuda.detect()

# Prueba forall()

In [ ]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def f(r_d_x,r_d_y,r_d_z,v_d_x,v_d_y,v_d_z,dr_d_x,dr_d_y,dr_d_z,dv_d_x,dv_d_y,dv_d_z):
  i = cuda.grid(1)
  size = len(r_d_x)
  if i < size:
    dr_d_x[i] = v_d_x[i]
    dr_d_y[i] = v_d_y[i]
    dr_d_z[i] = v_d_z[i]

    for j in range(size):
      if j!=i:
        dist_x = r_d_x[j] - r_d_x[i]
        dist_y = r_d_y[j] - r_d_y[i]
        dist_z = r_d_z[j] - r_d_z[i]

        norm_dist = math.sqrt(dist_x**2+dist_y**2+dist_z**2)

        dv_d_x[i] += (dist_x)/(norm_dist**3) # *m_j*G
        dv_d_y[i] += (dist_y)/(norm_dist**3) # *m_j*G
        dv_d_z[i] += (dist_z)/(norm_dist**3) # *m_j*G


Nb = 4 # nº cuerpos
Nc = 3 # nº coord

U0 = np.zeros(Nb*2*Nc)
U_0 = np.reshape(U0, (Nb, 2, Nc) )
r0 = np.reshape(U_0[:, 0, :], (Nb, Nc))
v0 = np.reshape(U_0[:, 1, :], (Nb, Nc))

# body 1 
r0[0,:] = [ 2., 2., 0.]
v0[0,:] = [ -0.4, 0., 0.]

# body 2 
r0[1,:] = [ -2., 2., 0.] 
v0[1,:] = [ 0., -0.4, 0.]

# body 3 
r0[2, :] = [ -2., -2., 0. ] 
v0[2, :] = [ 0.4, 0., 0. ] 
         
# body 4 
r0[3, :] = [ 2., -2., 0. ] 
v0[3, :] = [ 0., 0.4, 0. ]

U_sol = np.reshape(U0,(Nb,2,Nc)) #Reshape de la U: dividir en arrays para cada cuerpo (i), cada cuerpo tiene en fila 0 la r(i) y en fila 1 la v(i)
F = np.zeros(len(U0), dtype=np.float64)
#F = np.array([ [-0.4,0.,0.,-0.08459709, -0.08459709 ,0.],[0.,-0.4,0.,0.08459709, -0.08459709, 0.], [0.4,0.,0.,0.08459709,0.08459709,0.], [0.,0.4,0.,-0.08459709,0.08459709,0.] ])
dU_sol = np.reshape(F, (Nb, 2, Nc)) # Los valores introducidos en dU_sol serán insertados en F

r = np.reshape(U_sol[:, 0, :], (Nb, Nc)) # Guarda en array posiciones cada uno de los r(i) --> POSICIONES
v = np.reshape(U_sol[:, 1, :], (Nb, Nc)) # Guarda en array velocidades cada uno de los v(i) --> VELOCIDADES

r_x = r[:,0]
r_y = r[:,1]
r_z = r[:,2]
v_x = v[:,0]
v_y = v[:,1]
v_z = v[:,2]

drdt = np.reshape(dU_sol[:, 0, :], (Nb, Nc)) # Guarda en array derivada posiciones cada uno de los dr(i)/dt --> VELOCIDADES
dvdt = np.reshape(dU_sol[:, 1, :], (Nb, Nc)) # Guarda en array derivada velocidades cada uno de los dv(i)/dt --> ACELERACIONES

drdt_x = drdt[:,0]
drdt_y = drdt[:,1]
drdt_z = drdt[:,2]
dvdt_x = dvdt[:,0]
dvdt_y = dvdt[:,1]
dvdt_z = dvdt[:,2]

r_d_x = cuda.to_device(np.ascontiguousarray(r_x, dtype=np.float64))
v_d_x = cuda.to_device(np.ascontiguousarray(v_x, dtype=np.float64))
dr_d_x = cuda.to_device(np.ascontiguousarray(drdt_x, dtype=np.float64))
dv_d_x = cuda.to_device(np.ascontiguousarray(dvdt_x, dtype=np.float64))

r_d_y = cuda.to_device(np.ascontiguousarray(r_y, dtype=np.float64))
v_d_y = cuda.to_device(np.ascontiguousarray(v_y, dtype=np.float64))
dr_d_y = cuda.to_device(np.ascontiguousarray(drdt_y, dtype=np.float64))
dv_d_y = cuda.to_device(np.ascontiguousarray(dvdt_y, dtype=np.float64))

r_d_z = cuda.to_device(np.ascontiguousarray(r_z, dtype=np.float64))
v_d_z = cuda.to_device(np.ascontiguousarray(v_z, dtype=np.float64))
dr_d_z = cuda.to_device(np.ascontiguousarray(drdt_z, dtype=np.float64))
dv_d_z = cuda.to_device(np.ascontiguousarray(dvdt_z, dtype=np.float64))

f.forall(len(r))(r_d_x,r_d_y,r_d_z,v_d_x,v_d_y,v_d_z,dr_d_x,dr_d_y,dr_d_z,dv_d_x,dv_d_y,dv_d_z)
drdt_x = dr_d_x.copy_to_host()
drdt_y = dr_d_y.copy_to_host()
drdt_z = dr_d_z.copy_to_host()
dvdt_x = dv_d_x.copy_to_host()
dvdt_y = dv_d_y.copy_to_host()
dvdt_z = dv_d_z.copy_to_host()

print(dvdt_x)

# Prueba número de bloques

In [ ]:
from numba import cuda
import numpy as np
import math

def random_vector(rand=np.random):
    '''
    Devuelve un vector unitario de coordenadas XYZ
    '''
    phi = rand.uniform(0,2*np.pi)
    theta = rand.uniform(0,2*np.pi)
    x = np.cos(theta)*np.sin(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(phi)
    vec = np.array([x,y,z])
    return vec

def random_mass(rand=np.random):
    '''
    Devuelve la masa del cuerpo [kg]
    '''
    return rand.uniform(4e5,2e30)

def random_dist(rand=np.random):
    return rand.uniform(46e6,6e9)

@cuda.jit
def f(r_d_x,r_d_y,r_d_z,v_d_x,v_d_y,v_d_z,dr_d_x,dr_d_y,dr_d_z,dv_d_x,dv_d_y,dv_d_z):
  i = cuda.grid(1)
  size = len(r_d_x)
  if i < size:
    dr_d_x[i] = v_d_x[i]
    dr_d_y[i] = v_d_y[i]
    dr_d_z[i] = v_d_z[i]

    for j in range(size):
      if j!=i:
        dist_x = r_d_x[j] - r_d_x[i]
        dist_y = r_d_y[j] - r_d_y[i]
        dist_z = r_d_z[j] - r_d_z[i]

        norm_dist = math.sqrt(dist_x**2+dist_y**2+dist_z**2)

        dv_d_x[i] += (dist_x)/(norm_dist**3) # *m_j*G
        dv_d_y[i] += (dist_y)/(norm_dist**3) # *m_j*G
        dv_d_z[i] += (dist_z)/(norm_dist**3) # *m_j*G


Nb = 1024*72*1 # nº cuerpos tiene que ser 1024 (nº threads por bloque) * 72 (nº bloques o SM) * A (nº entero a elegir)
Nc = 3 # nº coord

U0 = np.zeros(Nb*2*Nc)
U_0 = np.reshape(U0, (Nb, 2, Nc) )
r0 = np.reshape(U_0[:, 0, :], (Nb, Nc))
v0 = np.reshape(U_0[:, 1, :], (Nb, Nc))

# Condiciones iniciales random
for k in range(Nb):
  r0[k,:] = random_vector()
  v0[k,:] = random_vector()


U_sol = np.reshape(U0,(Nb,2,Nc)) #Reshape de la U: dividir en arrays para cada cuerpo (i), cada cuerpo tiene en fila 0 la r(i) y en fila 1 la v(i)
F = np.zeros(len(U0), dtype=np.float64)
#F = np.array([ [-0.4,0.,0.,-0.08459709, -0.08459709 ,0.],[0.,-0.4,0.,0.08459709, -0.08459709, 0.], [0.4,0.,0.,0.08459709,0.08459709,0.], [0.,0.4,0.,-0.08459709,0.08459709,0.] ])
dU_sol = np.reshape(F, (Nb, 2, Nc)) # Los valores introducidos en dU_sol serán insertados en F

r = np.reshape(U_sol[:, 0, :], (Nb, Nc)) # Guarda en array posiciones cada uno de los r(i) --> POSICIONES
v = np.reshape(U_sol[:, 1, :], (Nb, Nc)) # Guarda en array velocidades cada uno de los v(i) --> VELOCIDADES

r_x = r[:,0]
r_y = r[:,1]
r_z = r[:,2]
v_x = v[:,0]
v_y = v[:,1]
v_z = v[:,2]

drdt = np.reshape(dU_sol[:, 0, :], (Nb, Nc)) # Guarda en array derivada posiciones cada uno de los dr(i)/dt --> VELOCIDADES
dvdt = np.reshape(dU_sol[:, 1, :], (Nb, Nc)) # Guarda en array derivada velocidades cada uno de los dv(i)/dt --> ACELERACIONES

drdt_x = drdt[:,0]
drdt_y = drdt[:,1]
drdt_z = drdt[:,2]
dvdt_x = dvdt[:,0]
dvdt_y = dvdt[:,1]
dvdt_z = dvdt[:,2]

r_d_x = cuda.to_device(np.ascontiguousarray(r_x, dtype=np.float64))
v_d_x = cuda.to_device(np.ascontiguousarray(v_x, dtype=np.float64))
dr_d_x = cuda.to_device(np.ascontiguousarray(drdt_x, dtype=np.float64))
dv_d_x = cuda.to_device(np.ascontiguousarray(dvdt_x, dtype=np.float64))

r_d_y = cuda.to_device(np.ascontiguousarray(r_y, dtype=np.float64))
v_d_y = cuda.to_device(np.ascontiguousarray(v_y, dtype=np.float64))
dr_d_y = cuda.to_device(np.ascontiguousarray(drdt_y, dtype=np.float64))
dv_d_y = cuda.to_device(np.ascontiguousarray(dvdt_y, dtype=np.float64))

r_d_z = cuda.to_device(np.ascontiguousarray(r_z, dtype=np.float64))
v_d_z = cuda.to_device(np.ascontiguousarray(v_z, dtype=np.float64))
dr_d_z = cuda.to_device(np.ascontiguousarray(drdt_z, dtype=np.float64))
dv_d_z = cuda.to_device(np.ascontiguousarray(dvdt_z, dtype=np.float64))

#f.forall(len(r))(r_d_x,r_d_y,r_d_z,v_d_x,v_d_y,v_d_z,dr_d_x,dr_d_y,dr_d_z,dv_d_x,dv_d_y,dv_d_z)
nthreads = 512
nblocks = (len(r_d_x)//nthreads)+1
print('Number of blocks: ',nblocks)
f[nblocks,nthreads](r_d_x,r_d_y,r_d_z,v_d_x,v_d_y,v_d_z,dr_d_x,dr_d_y,dr_d_z,dv_d_x,dv_d_y,dv_d_z)

drdt_x = dr_d_x.copy_to_host()
drdt_y = dr_d_y.copy_to_host()
drdt_z = dr_d_z.copy_to_host()
dvdt_x = dv_d_x.copy_to_host()
dvdt_y = dv_d_y.copy_to_host()
dvdt_z = dv_d_z.copy_to_host()

drdt[:,0] = drdt_x
drdt[:,1] = drdt_y
drdt[:,2] = drdt_z
dvdt[:,0] = dvdt_x
dvdt[:,1] = dvdt_y
dvdt[:,2] = dvdt_z



Ejecución de rk4

In [ ]:

RK4_Scheme(U_0,0.001,15,f)

def RK4_Scheme(U, dt, t, F):

    k1 = F(U,t)
    k2 = F(U + k1 * dt/2, t + dt/2)
    k3 = F(U + k2 * dt/2, t + dt/2)
    k4 = F(U + k3 * dt, t + dt)

    return U + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)